# 全人類がわかるディープラーニング Day6演習
## 概要
本演習では以下の３つのモデルの実装ファイルに基づいた問題を出題する。

- "VAE.ipynb"：kerasを用いたVAEによるmnist手書き文字生成の実装
- "DCGAN.ipynb"：kerasを用いたDCGANによるmnist手書き文字生成の実装
- '"gomoku/learn.ipynb"'：tensorflowを用いた五目並べの強化学習の実装

いずれについても、全体の流れを確認した上で本演習を解答してください。

なお、予め用意されたコード（はそのまま使用し、指示された穴埋め部を編集してください。<br>
演習問題は全12問です。解答が必要な部分は######と表記されています。<br>
最後まで実行されることを確認して提出してください。<br>
最後にそれぞれの問題が正しいかチェックすることが可能です。<br>
問題文の意味がわからない、実行エラーが直らない場合はご相談ください。

所要時間：1~2時間

なお、VAE、DCGAN、強化学習の**pythonによる実装**はE資格の試験範囲には含まれない。(2020/8以降は不明)

In [1]:
# ライブラリのインポート
import numpy as np

# 乱数シードを指定
np.random.seed(seed=0)

## 1.VAE

### 問1-1

encoderの入力の形状(shape)を配列で答えなさい。ただし、入力画像は一枚とし、バッチは考慮しない（以下同様）。
ヒント：
```
# エンコーダー(画像→潜在変数)
input_img = keras.Input(shape=img_shape)

```
答え方の例：
```
encoder_dim = [100, 2]
```

In [2]:
encoder_dim = [28,28,1]######

### 問1-2

zの形状(shape)を配列で答えなさい。ヒント：
```
z = layers.Lambda(sampling)([z_mean, z_log_var])
```

In [3]:
z_shape = [2]######

### 問1-3
z_mean = [0, 1], z_log_var = [-0.1, 0.1]のとき、ランダムなzを一つ出力せよ。ヒント：

```
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim),
                              mean=0., stddev=1.) # Kerasでは一貫して最初の次元がサンプル数(バッチサイズ)
    return z_mean + K.exp(0.5*z_log_var) * epsilon
```
numpyでガウス分布を出力する関数を使用します。

In [4]:
def sampling(args):
  z_mean, z_log_var = args
  epsilon = np.random.normal(loc=0,scale=1.0,size=2)######)
  return z_mean+np.exp(0.5*z_log_var)*epsilon######

z_mean = np.array([0, 1])
z_log_var = np.array([-0.1, 0.1])
z = sampling([z_mean, z_log_var])
print(z)

[1.6780185  1.42067371]


### 問1-4

z_decodedの形状(shape)を配列で答えなさい。ヒント：
```
# 潜在空間のzをデコードしたz_decodedを定義
z_decoded = decoder(z)
```

In [5]:
z_decoded_shape = [28,28,1]######

## 2.DCGAN

### 問2-1

生成器の入力の形状(shape)を配列で答えなさい。ただし、入力画像は一枚とし、バッチは考慮しない（以下同様）。ヒント：
```
generator_input = keras.Input(shape=(latent_dim,))
```

In [6]:
generator_input_shape = [100]######

### 問2-2

生成器の出力の形状(shape)を配列で答えなさい。ヒント：
```
#28x28 64-channelsを28x28 1-channelsの特徴マップへ変換
#最後の出力の活性化関数はtanhを使う
x = layers.Conv2D(channels, 5, activation='tanh', padding='same')(x)
generator = keras.models.Model(generator_input, x)
```

In [7]:
generator_output_shape = [28,28,1]######

### 問2-3

識別器の入力の形状(shape)を配列で答えなさい。ヒント：
```
discriminator_input = layers.Input(shape=(height, width, channels))
```

In [8]:
discriminator_input_shape = [28,28,1]######

### 問2-4

識別器の出力の形状(shape)を配列で答えなさい。ヒント：
```
...
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1, activation='sigmoid')(x)

discriminator = keras.models.Model(discriminator_input, x)
```

In [9]:
discriminator_output_shape = [1]######

### 問2-5

生成器の学習時変数gan_outputの形状(shape)を配列で答えなさい。ヒント：
```
gan_output = discriminator(generator(gan_input))
```

In [10]:
gan_output_shape = [1]######

## 3.強化学習

### 問3-1

value_networkの入力の形状(shape)を配列で答えなさい。ただし、入力盤面は一つとし、バッチは考慮しない（以下同様）。ヒント：gomoku/value_network.py
```
x_image = tf.reshape(x, [-1, size, size, 2])    # [None, size, size, 2]
```

In [11]:
x_image_shape = [9,9,2]######


### 問3-2

黒が勝利したとき、初期盤面(黒手番)の報酬を答えなさい(スカラー値)。ヒント：gomoku/learn.py
```
# 報酬（黒勝利：1.0, 白勝利：-1.0, 引き分け：0）
q_value = [0.0] if win is 0 else [1.0] if win is 1 else [-1.0]
```

In [12]:
value = 1.0######


### 問3-3

探索された盤面から、AIによる着手を選択するとき、常に評価値valueが最大となる盤面を選択するのではなく、評価値に応じて確率的盤面を選択している。

強化学習においてこの行動を選択する方策はさまざまなものが存在するが、ここではソフトマックス法を使用している。

温度をTとし、３つの盤面の評価が以下の配列vで表されるとき、それぞれが選択される確率配列をnp.ndarrayで出力せよ。

In [13]:
v = np.array([-1, 0, 0.1])
t = 0.1

def softmax(x, T):
    u = np.sum(np.exp(x/T))######
    return np.exp(x/T)/u######

p = softmax(v, t)
print(p)

[1.22097726e-05 2.68938138e-01 7.31049653e-01]


## 正誤チェック

以下は自動正誤チェックになります。最後まで実行して回答が正しいかをチェックしてください。<br>
※ コードを変更しないでください。

In [14]:
answers = {
    '問1-1': encoder_dim, 
    '問1-2': z_shape,
    '問1-3': z,
    '問1-4': z_decoded_shape,
    '問2-1': generator_input_shape,
    '問2-2': generator_output_shape,
    '問2-3': discriminator_input_shape,
    '問2-4': discriminator_output_shape,
    '問2-5': gan_output_shape,
    '問3-1': x_image_shape,
    '問3-2': value, 
    '問3-3': p
}
# ハッシュ化
import hashlib
right_answers = {"\u554f1-1": "4537c50f8e7d34885e1b358138252bfe", "\u554f1-2": "beb4dbf9af069aa2df7b147229965085", "\u554f1-3": "0f0844189a7150d422ea84cfb4b1557a", "\u554f1-4": "4537c50f8e7d34885e1b358138252bfe", "\u554f2-1": "5dd14615efeb2d086e519ed35efd3f73", "\u554f2-2": "4537c50f8e7d34885e1b358138252bfe", "\u554f2-3": "4537c50f8e7d34885e1b358138252bfe", "\u554f2-4": "35dba5d75538a9bbe0b4da4422759a0e", "\u554f2-5": "35dba5d75538a9bbe0b4da4422759a0e", "\u554f3-1": "a6a31c4ce00ea3068fdd889ef96586ef", "\u554f3-2": "e4c2e8edac362acab7123654b9e73432", "\u554f3-3": "f1a80f9732fbda36a19909aad40214e0"}
complete = True
print('問題\t正誤\tあなたの回答')
print('--------------------------------------')
for key in answers:
    if right_answers[key] == hashlib.md5(str(answers[key]).encode('utf-8')).hexdigest():
        print(key, '\t正解\t', answers[key])
    else:
        print(key, '\t不正解\t', answers[key])
        complete = False
if complete:
    print('\n全問正解です。このファイルを提出してください。')
else:
    print('\n不正解の問題を訂正してください。')

問題	正誤	あなたの回答
--------------------------------------
問1-1 	正解	 [28, 28, 1]
問1-2 	正解	 [2]
問1-3 	正解	 [1.6780185  1.42067371]
問1-4 	正解	 [28, 28, 1]
問2-1 	正解	 [100]
問2-2 	正解	 [28, 28, 1]
問2-3 	正解	 [28, 28, 1]
問2-4 	正解	 [1]
問2-5 	正解	 [1]
問3-1 	正解	 [9, 9, 2]
問3-2 	正解	 1.0
問3-3 	正解	 [1.22097726e-05 2.68938138e-01 7.31049653e-01]

全問正解です。このファイルを提出してください。
